# Bernstein-Vazirani alrogithm 

Given function $$f: \{0, 1\}^{n} \rightarrow \{0, 1\}$$
The goal is to determine: $$ f (x) = a \bullet x $$
where $$ a \bullet x = a_0 x_0 \oplus a_1 x_1 \oplus \dots \oplus a_{n_1} x_{n_1} $$

In [1]:
import numpy as np
import quant
from functools import reduce

ModuleNotFoundError: No module named 'quant'

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1
Create input vector 
$$ | 0 \dots 0 \rangle \oplus | 1 \rangle$$

In [4]:
qubit_number = 3
a = 7

In [5]:
x_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], qubit_number, axis=0))
y1 = quant.ONE
input_vector = np.kron(x_0n, y1)

NameError: name 'reduce' is not defined

## Step 2
Apply Hadamard $H^{\oplus n+1}$

In [6]:
H = quant.hadamard_n(qubit_number + 1)

NameError: name 'quant' is not defined

In [7]:
vector_s2 = np.dot(H, input_vector)

NameError: name 'H' is not defined

## Step 3
Apply function $U_f$


In [8]:
def create_bernstein_vazirani_function(qubit_number, func):
    states = 2 ** qubit_number
    res = np.empty([states, states])
    for num in range(states):
        r = func(num)
        s = format(r, '0' + str(states) + 'b')
        column = np.array(list(s), dtype=int)
        res[:, num] = column
    
    return res

In [9]:
U_f = create_bernstein_vazirani_function(qubit_number + 1, lambda x: quant.bit_count(a & x) & 1)

NameError: name 'quant' is not defined

In [10]:
vector_s3 = np.dot(U_f, vector_s2)

NameError: name 'U_f' is not defined

## Step 4
Apply Hadamard to input vectors

In [11]:
H_n = quant.hadamard_n(qubit_number)

NameError: name 'quant' is not defined

In [12]:
vector_s4 = np.dot(np.kron(H_n, np.eye(2)), vector_s3)

NameError: name 'H_n' is not defined

## Step 5
Measuring stage

In [13]:
for basis in range(2**qubit_number):
    quant.measure(vector_s4, basis)

NameError: name 'quant' is not defined